# Home task: pandas 

In [221]:
import pandas as pd

### Question 1

In [222]:
def answer_one():

    # Load energy data from 'Energy Indicators.xls' (which was provided in the repository), skipping header and footer
    energy = pd.read_excel('Energy Indicators.xls', skiprows = 17, skipfooter = 38)

    # Drop the first two columns
    energy = energy.iloc[:, 2:]

    # Rename columns
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    # Convert missing data to np.NaN
    # Since missing values appear only in columns that are supposed to be numeric, use pd.to_numeric()
    for column in energy.columns[1:]:
        energy[column] = pd.to_numeric(energy[column], errors = 'coerce')
    
    # Convert Energy Supply to gigajoules
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    # Remove digits and text in parentheses from  some country names
    energy['Country'] = energy['Country'].str.replace('\\d','', regex = True).str.replace(' [(].*', '', regex=True)

    # Rename specific countries
    countries_mapping = {
    'Republic of Korea': 'South Korea',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'China, Hong Kong Special Administrative Region': 'Hong Kong'
    }
    energy['Country'] = energy['Country'].replace(countries_mapping)

    # Load GDP data and skip header
    GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_19294.csv', skiprows = 4)

    # Rename specific countries
    countries_mapping_gdp = {    
    'Korea, Rep.': 'South Korea',
     'Iran, Islamic Rep.': 'Iran',
     'Hong Kong SAR, China': 'Hong Kong'}
    
    GDP['Country Name'] = GDP['Country Name'].replace(countries_mapping_gdp)

    # Load the "Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology"
    ScimEn = pd.read_excel('scimagojr.xlsx')

    # Keep only GDP columns for 2006-2015 and the 'Country' column
    for column in GDP.columns:
        if (column.isdigit() and (int(column) < 2006 or int(column) > 2015)) or (not column.isdigit() and column != 'Country Name') :
            GDP = GDP.drop(columns = column)

    # Merge the three datasets: ScimEn (top 15), energy, and GDP
    GDP = GDP.rename(columns={'Country Name': 'Country'})
    
    merged = pd.merge(
    pd.merge(ScimEn.iloc[:15], energy, on='Country'), 
    GDP, 
    on='Country'
    )

    # Drop 'Region' column, as it's not part of the expected output
    merged = merged.drop(columns = 'Region')

    # Set country names as index
    merged = merged.set_index(['Country'])
    return merged

In [223]:
df = answer_one()
print('Shape of the result DataFrame: ',df.shape)
df.head()

Shape of the result DataFrame:  (15, 20)


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,273437,272374,2336764,1615239,8.55,245,1.271910e+11,93.0,19.75491,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551546e+12,8.532185e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,2,175891,172431,2230544,724472,12.68,363,9.083800e+10,286.0,11.57098,1.381558e+13,1.447423e+13,1.476986e+13,1.447807e+13,1.504897e+13,1.559973e+13,1.625397e+13,1.688068e+13,1.760814e+13,1.829502e+13
India,3,55082,53775,463165,162944,8.41,181,3.319500e+10,26.0,14.96908,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856722e+12,2.039126e+12,2.103588e+12
Japan,4,50523,50065,488062,119930,9.66,193,1.898400e+10,149.0,10.23282,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,43389,42284,615670,111290,14.19,226,7.920000e+09,124.0,10.60047,2.708442e+12,3.090510e+12,2.929412e+12,2.412840e+12,2.485483e+12,2.663806e+12,2.707090e+12,2.784854e+12,3.064708e+12,2.927911e+12


### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [224]:
def answer_two():
    Top15 = answer_one()
    avgGDP = Top15.loc[:, '2006':].mean(axis=1).sort_values(ascending=False) # mean() method already ignores NaN values by default
    return avgGDP

In [225]:
answer_two()

Country
United States         1.572243e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.590729e+12
United Kingdom        2.777505e+12
France                2.692000e+12
Italy                 2.152983e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.406644e+12
South Korea           1.221328e+12
Australia             1.207997e+12
Iran                  4.567516e+11
dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [226]:
def answer_three():
    Top15 = answer_one()

    return Top15.loc[answer_two().index[5], '2015'] - Top15.loc[answer_two().index[5], '2006']

In [227]:
answer_three()

124621907951.68018

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [228]:
def answer_four():
    Top15 = answer_one()
    cit_ratio = (Top15['Self-citations']/Top15['Citations']).sort_values(ascending=False)

    return (cit_ratio.index[0],cit_ratio.iloc[0])


In [229]:
answer_four()


('China', 0.6912289816173135)

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [230]:
def answer_five():
    Top15 = answer_one()
    population = (Top15['Energy Supply']/Top15['Energy Supply per Capita']).sort_values(ascending=False)
    return population.index[2]

In [231]:
answer_five()

'United States'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [169]:
def answer_six():
    Top15 = answer_one()
    Top15['Citable Documents per Capita'] = Top15['Citable documents']/(Top15['Energy Supply']/Top15['Energy Supply per Capita'])
    
    return Top15[['Citable Documents per Capita','Energy Supply per Capita']].corr().iloc[0,1] # Pearson's correlation is a default method


In [232]:
answer_six()

0.7434709127726777

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_seven():
    Top15 = answer_one()
    
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

    Top15['Continent'] = Top15.index.to_series().replace(ContinentDict)

    continet_population = Top15.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])
    continet_population['std'] = continet_population['std'].fillna(0)
    
    return continet_population

In [217]:
answer_seven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,0.000000e+00
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,0.000000e+00
